<a href="https://colab.research.google.com/github/geeneelair/-09-313/blob/main/%D0%94%D0%97_%D0%9E%D0%9E%D0%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

15 Вариант (Учет и управление спортивным клубом)

In [ ]:
import json
import logging
from abc import ABC, abstractmethod, ABCMeta
from datetime import datetime
from dateutil.parser import parse
from typing import Union, Optional, List, Dict, Type


class PermissionDeniedError(Exception):
    pass


def check_permissions(permission: str):
    """
    Декоратор для проверки прав доступа.

    Аргументы:
        permission: имя права, которое должно быть у участника.
    """
    def decorator(func):
        def wrapper(self, *args, **kwargs):
            actor = args[0] if args else None
            if not actor or permission not in getattr(actor, 'permissions', []):
                raise PermissionDeniedError(f"Нет прав на действие '{permission}'")
            return func(self, *args, **kwargs)
        return wrapper
    return decorator


class MemberMeta(ABCMeta):
    """
    Метакласс для автоматической регистрации подклассов Member.

    Подклассы добавляются в реестр для динамического создания.
    """
    registry: Dict[str, Type] = {}

    def __init__(cls, name, bases, attrs):
        super().__init__(name, bases, attrs)
        if name != 'Member':
            MemberMeta.registry[name.lower()] = cls


class InvalidMemberError(Exception):
    pass


class SessionFullError(Exception):
    pass


class Bookable(ABC):
    """
    Интерфейс, определяющий возможность бронирования сессии.
    """
    @abstractmethod
    def book_session(self, participant):
        pass


class Reportable(ABC):
    """
    Интерфейс, определяющий возможность генерации отчетов.
    """
    @abstractmethod
    def generate_report(self):
        pass


class LoggingMixin:
    """
    Миксин, добавляющий возможность логирования действий.
    """
    logger = logging.getLogger('club_logger')

    def log_action(self, action: str):
        self.logger.info(action)


class NotificationMixin:
    """
    Миксин, добавляющий возможность отправки уведомлений.
    """
    @staticmethod
    def send_notification(message: str):
        print(f"Notification: {message}")


class Member(ABC, metaclass=MemberMeta):
    """
    Абстрактный класс участника (Member) клуба.

    Атрибуты:
        member_id: уникальный идентификатор
        name: имя
        age: возраст
        membership_type: тип членства
        join_date: дата вступления
        permissions: список прав доступа
    """
    _id_counter = 1

    def __init__(self, name: str, age: int, membership_type: str, join_date: str):
        if not name or age <= 0 or not membership_type:
            raise InvalidMemberError('Некорректные данные участника')
        self._member_id = Member._id_counter
        Member._id_counter += 1
        self._name = name
        self._age = age
        self._membership_type = membership_type
        self._join_date = parse(join_date)
        self.permissions: List[str] = ['book_session']

    @abstractmethod
    def get_membership_info(self) -> str:
        pass

    @property
    def member_id(self) -> int:
        return self._member_id

    @property
    def name(self) -> str:
        return self._name

    @name.setter
    def name(self, value: str):
        if not value:
            raise InvalidMemberError('Имя не может быть пустым')
        self._name = value

    @property
    def age(self) -> int:
        return self._age

    @age.setter
    def age(self, value: int):
        if value <= 0 or value > 140:
            raise InvalidMemberError('Недопустимый возраст')
        self._age = value

    @property
    def membership_type(self) -> str:
        return self._membership_type

    @membership_type.setter
    def membership_type(self, value: str):
        if not value:
            raise InvalidMemberError('Тип членства не может быть пустым')
        self._membership_type = value

    @property
    def join_date(self) -> datetime:
        return self._join_date

    @join_date.setter
    def join_date(self, value: Union[str, datetime]):
        if isinstance(value, str):
            self._join_date = parse(value)
        elif isinstance(value, datetime):
            self._join_date = value
        else:
            raise InvalidMemberError('Неверный формат даты')

    def __str__(self) -> str:
        return f"Член клуба: {self._name}, Тип членства: {self._membership_type}"

    def __eq__(self, other: object) -> bool:
        return isinstance(other, Member) and self._member_id == other._member_id

    def __lt__(self, other: 'Member') -> bool:
        return self._age < other._age

    def __gt__(self, other: 'Member') -> bool:
        return self._age > other._age

    def to_dict(self) -> Dict:
        data = {
            'type': self.__class__.__name__.lower(),
            'name': self._name,
            'age': self._age,
            'membership_type': self._membership_type,
            'join_date': self._join_date.isoformat()
        }
        if hasattr(self, '_subscription'):
            data['subscription'] = self._subscription
        if hasattr(self, '_specialization'):
            data['specialization'] = self._specialization
        return data

    @classmethod
    def from_dict(cls, data: Dict) -> 'Member':
        member_type = data.pop('type')
        return MemberFactory.create_member(member_type, **data)


class Client(Member):
    """
    Класс клиента клуба с дополнительным полем subscription.
    """
    def __init__(self, name: str, age: int, membership_type: str, join_date: str, subscription: str):
        super().__init__(name, age, membership_type, join_date)
        if not subscription:
            raise InvalidMemberError('Абонемент не может быть пустым')
        self._subscription = subscription

    def get_membership_info(self) -> str:
        return f"Клиент: {self._name}, Абонемент: {self._subscription}"

    @property
    def subscription(self) -> str:
        return self._subscription

    @subscription.setter
    def subscription(self, value: str):
        if not value:
            raise InvalidMemberError('Абонемент не может быть пустым')
        self._subscription = value

    def __str__(self) -> str:
        return self.get_membership_info()


class Trainer(Member):
    """
    Класс тренера клуба с дополнительным полем specialization.
    """
    def __init__(self, name: str, age: int, membership_type: str, join_date: str, specialization: str):
        super().__init__(name, age, membership_type, join_date)
        if not specialization:
            raise InvalidMemberError('Специализация не может быть пустой')
        self._specialization = specialization
        self.permissions.append('extend_membership')

    def get_membership_info(self) -> str:
        return f"Тренер: {self._name}, Специализация: {self._specialization}"

    @property
    def specialization(self) -> str:
        return self._specialization

    @specialization.setter
    def specialization(self, value: str):
        if not value:
            raise InvalidMemberError('Специализация не может быть пустой')
        self._specialization = value

    def __str__(self) -> str:
        return self.get_membership_info()


class Location:
    """
    Класс, описывающий место проведения тренировки.
    """
    def __init__(self, name: str, address: str):
        self.name = name
        self.address = address

    def __str__(self) -> str:
        return f"{self.name} ({self.address})"


class TrainingSession(Bookable, Reportable, LoggingMixin, NotificationMixin):
    """
    Класс тренировочной сессии.

    Поддерживает бронирование клиентов, логирование,
    уведомления и генерацию отчетов.
    """
    _session_counter = 1

    def __init__(self, trainer: Trainer, schedule: Union[str, datetime], location: Location, capacity: int = 10):
        self.session_id = TrainingSession._session_counter
        TrainingSession._session_counter += 1
        self.trainer = trainer
        self.schedule = parse(schedule) if isinstance(schedule, str) else schedule
        self.location = location
        self.capacity = capacity
        self.participants: List[Client] = []

    def add_participant(self, participant: Client):
        if len(self.participants) >= self.capacity:
            raise SessionFullError('Сессия переполнена')
        self.participants.append(participant)

    def remove_participant(self, participant: Client):
        self.participants.remove(participant)

    def get_participants(self) -> List[Client]:
        return list(self.participants)

    @check_permissions('book_session')
    def book_session(self, participant: Client):
        self.add_participant(participant)
        action = f"Участник {participant.name} записан на тренировку {self.session_id}"
        self.log_action(action)
        self.send_notification(f"{participant.name}, вы записаны на тренировку {self.session_id}")

    def generate_report(self) -> str:
        lines = [f"Отчет по тренировке {self.session_id}:",
                 f"Тренер: {self.trainer.name}",
                 f"Дата: {self.schedule}",
                 f"Место: {self.location}",
                 "Участники:"]
        lines += [f"- {p.name}" for p in self.participants]
        return "\n".join(lines)

    def to_dict(self) -> Dict:
        return {
            'session_id': self.session_id,
            'trainer': self.trainer.to_dict(),
            'participants': [p.to_dict() for p in self.participants],
            'schedule': self.schedule.isoformat(),
            'location': {'name': self.location.name, 'address': self.location.address},
            'capacity': self.capacity
        }

    @staticmethod
    def from_dict(data: Dict) -> 'TrainingSession':
        trainer = Member.from_dict(data['trainer'])
        participants = [Member.from_dict(pd) for pd in data['participants']]
        loc = Location(**data['location'])
        sess = TrainingSession(trainer, data['schedule'], loc, data.get('capacity', 10))
        sess.participants = participants
        return sess


class MemberFactory:
    """
    Фабрика для создания участников по типу.
    """
    @staticmethod
    def create_member(member_type: str, **kwargs) -> Member:
        cls = MemberMeta.registry.get(member_type.lower())
        if not cls:
            raise InvalidMemberError(f"Неизвестный тип участника: {member_type}")
        return cls(**kwargs)


class ExtensionHandler(ABC):
    """
    Базовый класс для обработчика в цепочке обязанностей (продление абонемента).
    """
    def __init__(self, successor: Optional['ExtensionHandler'] = None):
        self._successor = successor

    @abstractmethod
    def handle(self, months: int) -> str:
        pass

    def next(self, months: int) -> str:
        if self._successor:
            return self._successor.handle(months)
        raise PermissionDeniedError('Запрос не может быть обработан')


class Administrator(ExtensionHandler):
    """
    Обработчик для продления абонемента до 1 месяца.
    """
    def handle(self, months: int) -> str:
        if months <= 1:
            return f"Администратор одобрил продление на {months} мес."
        return self.next(months)


class Manager(ExtensionHandler):
    """
    Обработчик для продления абонемента до 3 месяцев.
    """
    def handle(self, months: int) -> str:
        if months <= 3:
            return f"Менеджер одобрил продление на {months} мес."
        return self.next(months)


class Director(ExtensionHandler):
    """
    Обработчик для продления абонемента на любое число месяцев.
    """
    def handle(self, months: int) -> str:
        return f"Директор одобрил продление на {months} мес."

class BookingProcess(ABC):
    """
    Шаблонный метод для процесса бронирования на тренировку.
    """
    def book_session(self, participant: Client, session: TrainingSession) -> None:
        self.check_availability(session)
        self.record_participant(participant, session)
        self.confirm_booking(participant, session)

    @abstractmethod
    def check_availability(self, session: TrainingSession) -> None:
        pass

    @abstractmethod
    def record_participant(self, participant: Client, session: TrainingSession) -> None:
        pass

    @abstractmethod
    def confirm_booking(self, participant: Client, session: TrainingSession) -> None:
        pass


class OnlineBookingProcess(BookingProcess):
    """
    Конкретная реализация бронирования онлайн.
    """
    def check_availability(self, session: TrainingSession) -> None:
        if len(session.participants) >= session.capacity:
            raise SessionFullError('Сессия переполнена')

    def record_participant(self, participant: Client, session: TrainingSession) -> None:
        session.participants.append(participant)

    def confirm_booking(self, participant:	Client, session: TrainingSession) -> None:
        print(f"[Online] {participant.name} успешно записан на сессию {session.session_id}")


class OfflineBookingProcess(BookingProcess):
    """
    Конкретная реализация бронирования оффлайн.
    """
    def check_availability(self, session: TrainingSession) -> None:
        if len(session.participants) >= session.capacity:
            raise SessionFullError('Сессия переполнена')

    def record_participant(self, participant:	Client, session: TrainingSession) -> None:
        session.participants.append(participant)

    def confirm_booking(self, participant:	Client, session: TrainingSession) -> None:
        print(f"[Offline] {participant.name} записан на сессию {session.session_id}")


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler('club.log', encoding='utf-8')]
)


if __name__ == '__main__':
    alice = MemberFactory.create_member('client', name='Alice', age=30, membership_type='Gold', join_date='2025-01-01', subscription='Monthly')
    bob = MemberFactory.create_member('trainer', name='Bob', age=40, membership_type='Staff', join_date='2024-06-15', specialization='Strength')
    hall = Location('Main Hall', '123 Fitness St.')
    session = TrainingSession(bob, '2025-05-01T10:00:00', hall)
    session.book_session(alice)
    print(session.generate_report())
    chain = Administrator(Manager(Director()))
    print(chain.handle(2))
    data = session.to_dict()
    print(json.dumps(data, ensure_ascii=False, indent=2))
